In [ ]:
#Scraping indeed website

In [1]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import urllib.request
import numpy as np

### Using Beautiful Soup to scrape indeed job postings, descripion on clicks, reviews from company's review page

In [2]:
link = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 1000
i = 0
results = []
df1 = pd.DataFrame(columns=["Title","Location","Company","Salary", "Summary", "Description", "Review"])
for city in set(['Atlanta',
 'Austin',
 'Baltimore',
 'Boston',
 'Charlotte',
 'Chicago',
 'Cincinnati',
 'Columbus',
 'Dallas',
 'Denver',
 'Houston',
 'Indianapolis',
 'Jacksonville',
 'Kansas+City',
 'Los+Angeles',
 'New+York',
 'Miami',
 'Minneapolis',
 'Nashville',
 'Oakland',
 'Philadelphia',
 'Phoenix',
 'Pittsburgh',
 'Portland',
 'San+Diego',
 'San+Francisco',
 'San+Jose',
 'Seattle',
 'St.+Louis',
 'Tomp',
 'Washington%2C+DC']):
    for start in range(0, max_results_per_city, 10):
        url = link.format(city, start)
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = None
            try:
                summary = each.find('span', {'class':'summary'}).text.replace('\n', '')
            except:
                summary = None
            try: 
                description_link = ("%s%s" % ("https://www.indeed.com",each.find('a').get('href')))
            except:
                description_link = 'None'
            try:
                review_link = ("%s%s" % ("https://www.indeed.com",each.find('div').find('a').get('href')))            
            except: 
                review_link = 'None'
            df1 = df1.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Summary':summary,\
                                      'Description':description_link, 'job_description': None, 'Review': review_link,\
                           'comp_rating_overall': None, 'wl_bal_rating': None, 'benefit_rating': None, 'jsecurity_rating': None,\
                            'mgmt_rating': None, 'culture_rating': None}, ignore_index=True)
    sleep(1)

In [6]:
df = pd.read_csv('jobs_desc_ratings.csv')

In [18]:
def get_jobdesc(df):
    for i in range(0,len(df)):
        company_name = df.iloc[i]['Company']
        desc = df.iloc[i]['Description']
        html = requests.get(desc)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        try:
            df.loc[df['Company'] == company_name, 'job_description'] = soup.find(class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text.replace('\n', ' ')
        except:
            df.loc[df['Company'] == company_name, 'job_description'] = 'None'
        sleep(0.10)
    return df

In [19]:
def get_jobdesc_again(df):
    for i in range(0,len(df)):
        if(df.iloc[i]['job_description'] == 'None'):
            company_name = df.iloc[i]['Company']
            desc = df.iloc[i]['Description']
            html = requests.get(desc)
            soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
            try:
                df.loc[df['Company'] == company_name, 'job_description'] = soup.find(class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text.replace('\n', ' ')
            except:
                df.loc[df['Company'] == company_name, 'job_description'] = 'None'
            sleep(0.10)
    return df

In [20]:
def get_reviews(df):
    for i in range(0,len(df)):
        if(df.iloc[i]['Review'] != 'None'):
            company_name = df.iloc[i]['Company']
            cmp_page = df.iloc[i]['Review']
            html = requests.get(cmp_page)
            soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
            try:
                df.loc[df['Company'] == company_name, 'comp_rating_overall'] = float(soup.find(class_="cmp-header-rating-average").text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'comp_rating_overall'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'wl_bal_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[0].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'wl_bal_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'benefit_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[1].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'benefit_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'jsecurity_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[2].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'jsecurity_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'mgmt_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[3].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'mgmt_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'culture_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[4].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'culture_rating'] = 'None'
            sleep(0.10)
    return df

In [ ]:
df.to_csv('jobs_desc_ratings.csv', index = False)
df =  pd.read_csv('jobs_desc_ratings.csv')

In [25]:
df = df.copy() 

In [26]:
df = get_jobdesc(df)
df = get_jobdesc_again(df)
df = get_reviews(df)

In [ ]:
df.to_csv('jobs_complete.csv', index = False)

In [76]:
def get_city_state(df,col):
    df = df.join(df[col].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))
    df['City'] = df['City'].astype(str)
    df['State'] = df['State'].astype(str)
    df['State'] = [x[0:3] for x in df['State'] if x != None]
    df.drop(['Location'], axis = 1, inplace = True)
    return df

In [77]:
df = get_city_state(df, 'Location')
df = df[['City', 'State']]
new_df = pd.DataFrame(df4.groupby('City')['alternate'].apply(set)).reset_index()
new_df.columns = ['City', 'center']
new_df['centre'] = new_df['center'].apply(lambda x: list(x))
new_df['centre1'] = [x[0] for x in new_df['centre']]
new_df = new_df[['City', 'centre1']]
new_df = new_df.replace('Washington%2C DC', 'Washington DC')
new_df.to_csv('city_list.csv', index = False)

In [38]:
city_df = pd.DataFrame(df4['City'].value_counts().index)
city_df.columns = ['cities']
city_df

In [183]:
city_df['new_city'] = df4.loc[(df4['City'] == city_df['cities']), 'alternate']
city_df.to_csv('city_list.csv', index = False)